In [20]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, Input, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adadelta
import numpy as np
import math



In [12]:
(x_train, y_train),(x_test, y_test) = cifar10.load_data()

print(x_train.shape)
# One hot encoding
num_classes = 10
y_test = to_categorical(y_test, num_classes)
y_train =  to_categorical(y_train, num_classes)

(50000, 32, 32, 3)


In [25]:
def build_conv_layers(prev_layer, filters, filter_size, regularizer=None, maxpooling=True):
    conv1 = Conv2D(filters, filter_size, padding='same', activation="relu", kernel_regularizer=regularizer)(prev_layer)
    batchNormal = BatchNormalization()(conv1)
    if maxpooling==True:
        pool = MaxPooling2D(pool_size=(2, 2))(batchNormal)
        return pool
    else:
        return batchNormal
    
    


In [24]:
def build_fc_layers(prev_layer, filters, activation="relu", add_Dropout=True):
    dense1 = Dense(filters, activation = activation)(prev_layer)
    batchNormal = BatchNormalization()(dense1)
    if add_Dropout==True:
        dropout = Dropout(0.5)(batchNormal)
        return dropout
    else:
        return batchNormal


In [29]:
l2_reg = 0.001

input_layer = Input(shape= x_train.shape[1:], name = 'input_layer')
conv_1 = build_conv_layers(input_layer, 96, (11,11), l2(l2_reg))
conv_2 = build_conv_layers(conv_1, 256, (5,5))
zeropad_1 = ZeroPadding2D(padding=1)(conv_2)
conv_3 = build_conv_layers(zeropad_1, 512, (3,3))
zeropad_2 = ZeroPadding2D(padding=1)(conv_3)
conv_4 = build_conv_layers(zeropad_2, 1024, (3,3), None, False)
zeropad_3 = ZeroPadding2D(padding=1)(conv_4)
conv_5 = build_conv_layers(zeropad_3, 1024, (3,3))
flatten = Flatten()(conv_5)
fc_1 = build_fc_layers(flatten, 3072)
fc_2 = build_fc_layers(fc_1, 4096)
fc_3 = build_fc_layers(fc_2, num_classes, "softmax", False)

model = Model(inputs=input_layer, outputs=fc_3)

model.summary()




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_8 (Conv2D)           (None, 32, 32, 96)        34944     
                                                                 
 batch_normalization_8 (Batc  (None, 32, 32, 96)       384       
 hNormalization)                                                 
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 16, 16, 96)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 256)       614656    
                                                                 
 batch_normalization_9 (Batc  (None, 16, 16, 256)      1024  

In [30]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              metrics = ['accuracy'])

In [31]:
# Training Parameters
batch_size = 64
epochs = 25

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

model.save("CIFAR10_AlexNet_10_Epoch.h5")

# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/25
  8/782 [..............................] - ETA: 1:17:33 - loss: 7.9012 - accuracy: 0.0859

KeyboardInterrupt: 